In [ ]:

import matplotlib.pyplot as plt
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
house.hist(bins=46,figsize=(12,10))
plt.show()










    


In [ ]:
house["med_inc_cat"] = pd.cut(house["median_income"], bins=[0, 1, 3, 4, 6, np.inf], labels=[1, 2, 3, 4, 5])

In [ ]:
house["med_inc_cat"].value_counts().sort_index().plot.bar(rot=0, figsize=(12,10),grid=True)
plt.title("Median Income")
plt.xlabel("income")
plt.ylabel("count")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

strat_train_set, strat_test_set = train_test_split(
house, test_size=0.2, stratify=house["med_inc_cat"], random_state=42)
print(strat_test_set["med_inc_cat"].value_counts() / len(strat_test_set))
house["med_inc_cat"].value_counts() / len(house)




    
    
    
    
    
    
    
    


In [ ]:
house=strat_train_set.copy()
print(strat_train_set.head())


In [ ]:
house.plot(kind="scatter",x="longitude",y="latitude",grid="True",alpha=0.2)
plt.show()

In [ ]:
house.plot(kind="scatter",x="longitude",y="latitude",grid="True",s=(house["population"])/1000,c=house["median_house_value"],cmap="jet",colorbar=True,legend=True
,figsize=(12,10))
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix
attributes=["median_house_value","population","median_income","total_rooms","latitude","longitude"]
scatter_matrix(house[attributes],figsize=(12,10))
                                               



In [ ]:
house.plot(kind="scatter",x="median_income",y="median_house_value",alpha=0.2,grid=True)
plt.show()

In [ ]:
house.plot(kind="scatter",x="housing_median_age",y="median_house_value",alpha=0.2,grid=True)
plt.show()

In [ ]:
house.plot(kind="scatter",x="housing_median_age",y="population",alpha=0.2,grid=True)
plt.show()

In [ ]:

house=strat_train_set.drop("median_house_value",axis=1)
print(house.shape)
housing_labels=strat_train_set["median_house_value"].copy()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder


from sklearn.compose import make_column_selector
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer



cat_pipeline=make_pipeline(SimpleImputer(strategy="most_frequent"),OneHotEncoder(handle_unknown="ignore"))






def ratio(x):
    return x[:,[0]]/x[:,[1]]
def ratio_name(feature_transformer,features_names_in):
    return ['ratio']

def ratio_pipeline():
    return make_pipeline(SimpleImputer(strategy="median"),FunctionTransformer(ratio,feature_names_out=ratio_name),StandardScaler())
log_pipeline=make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log,feature_names_out="one-to-one")
)
Cluster_Simil=KMeans(n_clusters=10,n_init=10,random_state=100)
default_num_pipeline=make_pipeline(SimpleImputer(strategy="median"),StandardScaler())
custom_trans=ColumnTransformer([
    ("pop_per_house",ratio_pipeline(),["population","households"]),
    ("rooms_per_house",ratio_pipeline(),["total_rooms","households"]),
    ("bedroom_per_room",ratio_pipeline(),["total_bedrooms","total_rooms"]),
    ("log",log_pipeline,["total_rooms","total_bedrooms","median_income","population"]),
    ("geo",Cluster_Simil,["longitude","latitude"]),
    ("cat",cat_pipeline,make_column_selector(dtype_include="object")),
    
    
    
    
],
   remainder=default_num_pipeline)
house_prepared=custom_trans.fit_transform(house)
print(house_prepared.shape)
    


    


        


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rtree_reg=make_pipeline(custom_trans,RandomForestRegressor(random_state=22))
rtree_reg.fit(house,housing_labels)
predict_rtree=rtree_reg.predict(house)



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
final_model=make_pipeline(custom_trans,RandomForestRegressor(max_features=6,random_state=88,n_estimators=500))
final_model.fit(house,housing_labels)



In [ ]:
from sklearn.metrics import mean_squared_error
prediction_test=final_model.predict(strat_test_set.drop("median_house_value",axis=1))
mse=mean_squared_error(strat_test_set["median_house_value"],prediction_test)
rmse=np.sqrt(mse)
print(rmse)
                                    

In [ ]:
from xgboost import XGBRegressor
xgb_final=make_pipeline(custom_trans,XGBRegressor(n_estimators=1000,max_depth=5,learning_rate=0.1,random_state=12))
xgb_final.fit(house,housing_labels)
pred_xg=xgb_final.predict(strat_test_set.drop("median_house_value",axis=1))
mse=mean_squared_error(strat_test_set["median_house_value"],pred_xg)
rmsse=np.sqrt(mse)
print(rmsse)



In [ ]:
import joblib
joblib.dump(rtree_reg,"housing_model.pkl")
print('SUCCESS')

In [ ]:
# from sklearn.svm import SVR
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import loguniform,expon
# from sklearn.pipeline import Pipeline
# 
# 
# param_distribs={
#     "svr__kernel":["rbf","linear"],
#     "svr__C":loguniform(2,200_000),
#     "svr__gamma":[expon(scale=1.0)],
#     
# }
# svr_pipeline=Pipeline([
#     ("preprocessing",custom_trans),
#     ("svr",SVR())
#     
#     
#     
#     
# ])
# random_search=RandomizedSearchCV(svr_pipeline,param_distributions=param_distribs,cv=3,n_iter=50,scoring="neg_mean_squared_error",random_state=44)
# random_search.fit(house,housing_labels)



In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

my_pipeline=Pipeline([
    ("preprocessing",custom_trans),
    ("svr",SVR()),
])
param_grid={
    "svr__kernel":["linear"],
    "svr__C":[2.,3.,40.,32.,38.,100.],






}
grid_search=GridSearchCV(my_pipeline,param_grid,cv=3,scoring="neg_mean_squared_error")
grid_search.fit(house[:5000],housing_labels[:5000])


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from scipy.stats import expon
param_distrib={
    "svr__kernel":["linear","rbf"],
    "svr__C":loguniform(2.,20_000),
    "svr__gamma":expon(scale=1.0),
    
    
}
rand_search=RandomizedSearchCV(my_pipeline,param_distrib,n_iter=10,cv=3,random_state=42,scoring="neg_mean_squared_error")
rand_search.fit(house[:5000],housing_labels[:5000])


    
    
    

    



In [ ]:
# from sklearn.preprocessing import SelectFromModel
# selector_pipeline=Pipeline([
#     ("preprocessing",custom_trans),
#     ("selector",SelectFromModel(RandomForestRegressor(random_state=44),threshold=0.001)),
#     ("svr",SVR(C=rnd.best_params_["svr__C"],
#                gamma=rnd.best_params_["svr__gamma"],
#                kernel=rnd.best_params_["svr__kernel"]))
#      
    
    
    
    

In [ ]:
# selector_rmses=-cross_val_score(selector_pipeline,house.iloc[:5000],housing_labels.iloc[:5000],scoring="neg_mean_squared_error",cv=3)
# # print(selector_rmses.describe())

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import MetaEstimatorMixin,clone,TransformerMixin,BaseEstimator
from sklearn.utils.validation import check_array,check_is_fitted

class FeatureFromRegressor(MetaEstimatorMixin,TransformerMixin,BaseEstimator):
    def __init__(self,estimator):
        self.estimator=estimator
    
    def fit(self, X, y = None):
        check_array(X)#input check
        self.estimator_=clone(self.estimator)
        self.estimator_.fit(X,y)
        self.n_features_in_=self.estimator_.n_features_in_
        if hasattr(self.estimator_,"features_names_in_"):
            self.features_names_in_=self.estimator_.feature_names_in_
        return self
    def transform(self,X):
        check_is_fitted(self)
        predictions=self.estimator_.predict(X)
        if predictions.ndim == 1:
            predictions=predictions.reshape(-1,1)
        return predictions
    def get_features_names_out(self,names=None):
        check_is_fitted(self)
        n_outputs=getattr(self.estimator,"n_outputs",1)
        estimator_class_name=self.estimator_.__class__.__name__
        estimator_short_name=estimator_class_name.lower().replace("_","")
        return[f"{estimator_short_name}_prediction_{i}"
           for i in range(n_outputs)]
    
        
    
    
    

                                                  




In [ ]:
from sklearn.utils.estimator_checks import check_estimator
test_results=check_estimator(FeatureFromRegressor(KNeighborsRegressor()))

In [ ]:
knn_reg=KNeighborsRegressor(n_neighbors=3,weights='distance')
knn_transformer=FeatureFromRegressor(knn_reg)
geo_features=house[["latitude","longitude"]]
knn_transformer.fit(geo_features,housing_labels)

In [ ]:
knn_transformer.get_features_names_out()


In [ ]:
from sklearn.base import clone
transformers= [(name,clone(transformer),columns)
               for name,transformer,columns in custom_trans.transformers]
geo_index=[name for name,_,_ in transformers].index("geo")
transformers[geo_index]=("geo",knn_transformer,["latitude","longitude"])
new_geo_preprocessing=ColumnTransformer(transformers)




In [ ]:
new_geo_pipeline=Pipeline([
    {"preprocessing",new_geo_preprocessing},
    {"svr",SVR(C=rand_search.best_params_,
    kernel=rand_search.best_params_,
               gamma=rand_search.best_params_)
            

                                                    }
    
    
    
])


In [ ]:
from sklearn.model_selection import cross_val_score
import pandas as pd
new_pipe_rmses=-cross_val_score(new_geo_pipeline,house.iloc[:5000],housing_labels.iloc[:5000],cv=3,scoring="neg_mean_squared_error")
pd.Series(new_pipe_rmses).describe()


In [ ]:
param_distrib={
    "preprocessing_geo_estimator_n_neighbors":range(1,30),
    "preprocessing_geo_estimator__weights":["distance","weights"],
    "svr__C":loguniform(20,200_000),
    "svr__gamma":expon(scale=1.0),
    
    
    
    
    
}
new_geo_rnd_search=RandomizedSearchCV(new_geo_pipeline,
                                      param_distributions=param_distrib,n_iter=50,scoring="neg_mean_squared_error",cv=3)
new_geo_rnd_search.fit(house.iloc[:5000],housing_labels.iloc[:5000])

                                      
                                      

In [ ]:
new_geo_rnd_search_rmse=-new_geo_rnd_search.best_score_
print(new_geo_rnd_search_rmse)

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.utils.validation import check_is_fitted,validate_data
class StandardScalerClone(TransformerMixin,BaseEstimator):
    def __init__(self,with_mean=None):
        self.with_mean=with_mean
    def fit(self,X,y=None):
        X=validate_data(self,X,ensure_2d=True)
        self.n_features_in_=X.shape[1]
        if self.with_mean:
            self.mean_=np.mean(X,axis=0)
        self.scale_=np.std(X,axis=0,ddof=0)
        self.scale_[self.scale_==0]=1
        return self
    def transform(self,X):
        check_is_fitted(self)
        X=validate_data(self,X,ensure_2d=True,reset=False)
        if self.with_mean:
            X=X-self.mean_
        return X/self.scale_
    def inverse_transform(self,X):
        check_is_fitted(self)
        X=validate_data(self,X,ensure_2d=True,reset=False)
        return X* self.scale_+self.mean_
    def get_features_names_out(self,input_features=None):
        if input_features is None:
            return getattr(self,"feature_names_in_",
                           [f"x{i}" for i in range(self.n_features_in_)])
        else:
            if len(input_features)!= self.n_features_in_:
                raise ValueError('invalid num of features')
            if hasattr(self,"features_names_in_") and not np.all(
                self.feature_names_in_ == input_features
                
            ):
                raise ValueError("input != feat name in")
            return input_features
                           
                           
                           


In [ ]:
from sklearn.utils.estimator_checks import check_estimator
check_estimator(StandardScalerClone())

In [ ]:
np.random.seed(42)


X=np.random.rand(1000,3)
scaler=StandardScalerClone()
X_scaled_uncentered=scaler.fit_transform(X)
assert np.allclose(X_scaled_uncentered,X/X.std(axis=0))

In [ ]:
scaler=StandardScalerClone()
X_back=scaler.inverse_transform(scaler.fit_transform(X))
assert np.allclose(X,X_back)


In [ ]:
assert np.all(scaler.get_feature_names_out()==["x0","x1","x2"])
assert np.all(scaler.get_feature_names_out(["a","b","c"])==["a","b","c"])

In [ ]:
df=pd.DataFrame({"a":np.random.rand(100),"b":np.random.rand(100)})
scaler=StandardScalerClone()
X_scaled=scaler.fit_transform(df)
assert np.all(scaler.feature_names_in_ ==["a","b"])
assert np.all(scaler.get_features_names_out()==["a","b"])